In [14]:
# --- Environment Fix ---
!pip install --force-reinstall "numpy<1.26" --quiet
!pip install --force-reinstall scikit-surprise==1.1.3 --no-cache-dir --quiet

import numpy as np, pandas as pd
print("✅ NumPy:", np.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 85.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 13.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error:

In [17]:
# =====================================================
# Online Course Recommendation System (NumPy 2.x Safe + Gradio Enhanced)
# =====================================================
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

# ---------- CONFIG ----------
file_path = "/content/online_course_recommendation.csv"  # Update path if needed
TOP_N_DEFAULT = 5

# ---------- Load dataset ----------
df = pd.read_csv(file_path)
print("✅ Dataset loaded:", df.shape)

# Add implicit rating if missing
if "rating" not in df.columns:
    df["rating"] = 1.0

# ---------- User-Course Matrix ----------
pivot = df.pivot_table(index="user_id", columns="course_id", values="rating", fill_value=0)

# ---------- SVD ----------
svd = TruncatedSVD(n_components=20, random_state=42)
user_factors = svd.fit_transform(pivot)
course_factors = svd.components_.T
course_ids = pivot.columns

# ---------- Cosine similarity ----------
similarity = cosine_similarity(course_factors)

# ---------- Recommendation Function ----------
def recommend_for(user_id, top_n=TOP_N_DEFAULT):
    user_id = str(user_id)
    if user_id not in pivot.index:
        return []

    user_vector = pivot.loc[user_id].values.reshape(1, -1)
    scores = user_vector.dot(similarity).flatten()

    # Remove already seen courses
    seen = set(df.loc[df.user_id == user_id, "course_id"])
    candidates = [(course_ids[i], scores[i]) for i in range(len(course_ids)) if course_ids[i] not in seen]

    # Top-N
    topn = sorted(candidates, key=lambda x: x[1], reverse=True)[:top_n]

    # Add metadata
    meta = df.drop_duplicates("course_id").set_index("course_id")
    recs = []
    for cid, score in topn:
        info = {"course_id": cid, "score": round(float(score), 4)}
        if cid in meta.index:
            row = meta.loc[cid]
            for col in ["description", "category", "tags"]:
                if col in row.index:
                    info[col] = row[col]
        recs.append(info)
    return recs

# ---------- Gradio Interface ----------
def recommend_ui(user_id, top_n):
    recs = recommend_for(user_id, top_n)
    if not recs:
        return pd.DataFrame([{"Error": f"No recommendations found. User '{user_id}' may be new."}])

    # Convert list of dicts to dataframe for clean display
    df_out = pd.DataFrame(recs)
    return df_out

iface = gr.Interface(
    fn=recommend_ui,
    inputs=[
        gr.Textbox(label="User ID", placeholder="Enter user_id, e.g., user_82"),
        gr.Slider(minimum=1, maximum=10, step=1, value=TOP_N_DEFAULT, label="Number of Recommendations")
    ],
    outputs=gr.Dataframe(label="Top-N Recommendations", type="pandas"),
    title="Online Course Recommender (Enhanced)",
    description="Enter a user ID to get personalized top-N course recommendations."
)

# Launch Gradio app
iface.launch(share=True)


✅ Dataset loaded: (300, 5)
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aa67735b929af9eba2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
